# 네이버 영화 평점 Crawling by 민또

### 기본적인 구조 파악

In [1]:
from bs4 import BeautifulSoup   ### 필요한 패키지 로드
import urllib.request as req
import math

In [2]:
url1 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='
num = 85579   ### 영화 고유 번호
url2 = '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='
page = 1   ### 페이지 번호 (한 페이지당 평점 10개)

response = req.urlopen(url1+str(num)+url2+str(page))
print(response)

In [3]:
soup = BeautifulSoup(response, 'html.parser')   # 소스 파싱
score_result = soup.find('div', class_ = 'score_result')   # 리뷰와 평점이 있는 부분만 자르기
lis = score_result.find_all('li')    # li 별로 잘라서 벡터형태로 변경

In [4]:
### 페이지 한 창의 평점 크롤링

for li in lis:  # 각 li 마다 평점과 코멘트 추출
    score = li.select_one('div.star_score > em')   ### 평점 추출
    reple = li.select_one('div.score_reple > p')   ### 리뷰 추출
    if reple.find('span')!=None:
        reple.find('span').decompose()   ### 리뷰앞에 붙은 '베스트', '관람객' 제거
    print('평점 :',  score.get_text())
    print('리뷰 :', reple.get_text(), '\n')

평점 : 10
리뷰 : 관람객이정재가 홍보 따라다닌 이유를 알겠음 ㅋㅋㅋ 그리고 관심병사 연기 도랐다   

평점 : 10
리뷰 : 관람객나 죽어서 지옥가면 어떡하냐 ㄷㄷㄷ 앞으로 착하게 살아야지   

평점 : 10
리뷰 : 관람객남자 셋이서 안 울라고…주먹으로 입틀막함 ㅋㅋㅋㅋ   

평점 : 10
리뷰 : 관람객여보, 이번 겨울에… 부모님댁에 밥솥 놔드려야겠어요   

평점 : 10
리뷰 : 관람객가족이랑 봤는데… 나… 우리 아빠 우는 거 처음 봤음 ㅜㅜㅜㅜ   

평점 : 8
리뷰 : 오늘도 열심히 사는 대한민국의 자홍,수홍씨들을 응원합니다 ㅠㅠ   

평점 : 10
리뷰 : 야 이거 생각한 것 보다 잘 나왔잖아 ?   

평점 : 10
리뷰 : 오늘 화장 안하고 신과함께를 본 것은 신의한수였음.   

평점 : 10
리뷰 : 원작팬이에요. 솔직히 기대 내려놓고 봤지만 볼거리, 스토리, 캐릭터 케미 모두 기대 이상이었어요. 이정도 영화 없다고 봅니다. 많이 감동 받았어요   

평점 : 10
리뷰 : 관람 내내 지옥행 롤러코스터 타고 온 기분이었음 ㅋㅋㅋㅋ   



### for 문을 사용하여 페이지와 영화종류를 바꿔가며 출력해보자!

In [5]:
url1 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='
movies = [85578, 136898]   ### 일단 두개의 영화만 돌릴거임~
url2 = '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='


for i in movies:
    num = i
    response = req.urlopen('https://movie.naver.com/movie/bi/mi/point.nhn?code='+str(num))   ### 영화제목
    soup = BeautifulSoup(response, 'html.parser')   ### 소스 파싱
    title =  soup.select_one('title').string   ### 영화 제목 추출(근데 봐서 이건 빼도 될듯...)
    print('###', title, '###', '\n')   ### 젤 첨에 영화 제목 출력
    
    for j in range(1, 4):   ### 1페이지부터 3페이지까지 돌림
        page = j
        response = req.urlopen(url1+str(num)+url2+str(page))
        soup = BeautifulSoup(response, 'html.parser')
        score_result = soup.find('div', class_ = 'score_result')
        lis = score_result.find_all('li')   # 리뷰가 담겨있는 각 마디를 잘라 벡터화
        
        for li in lis:
            score = li.select_one('div.star_score > em')
            reple = li.select_one('div.score_reple > p')
            if reple.find('span')!=None:
                reple.find('span').decompose()
            print(score.get_text(), ' , ', reple.get_text(), '\n')   ### 평점, 리뷰 출력

### 7년의 밤 : 네이버 영화 ### 

1  ,  염력도 피하고 골든슬럼버도 피했는데 이걸 못피함   

10  ,  장동건 원래 연기 잘 했죠.. 작품들이 안 좋아서 그렇지   

8  ,  배우들의 열연과 몰입도는 좋았지만 2시간 내내 똑같이 하드하기만 해서 힘들었음.. 완급을 주는 시나리오였음 좋았을듯   

9  ,  관람객장동건, 류승룡의 연기도 좋았지만 난 송새벽에게 한표 주고 싶음. 제 3자로써 이 이야기를 가장 많이 알고 있는 인물이었기 때문에 더 연기하기가 힘들었을텐데 정말 자연스럽게 잘하더라   

10  ,  관람객장동건이 연기를 위해 얼굴을 버렸네 그 열정에 박수!!   

4  ,  7년의밤이 아나라 2시간의밤임 2시간동안 밤처럼앞이 안보임   

9  ,  장동건 연기를 이렇게 잘하나 싶었다   

2  ,  무슨 얘기를 하자는 건지 태어나서 처음 평가한다 답답해서   

10  ,  세령마을 싱크 최고였어요ㅠ분위기와 배우들 연기로 압도   

10  ,  연기 몰입감 최고네 ㅎㄷㄷ;;   

9  ,  소설은 읽지 않았지만  연기 출연진 굿   

1  ,  7년 같았던 2시간..   

10  ,  조금 아쉽지만, 소설과 영화라는 매체의 차이를 고려했을때 나쁘지않은 각색이라고 생각합니다. 배우들의 묵직한 열연이 좋았어요!   

8  ,  장동건...연기 소름 오래전 읽었던 정유정 작가의 책을 다시금 펼쳐 읽어야겠다   

8  ,  책으로보고기대했는데 생각보다 긴장감도 있고재밌었습니다.   

10  ,  스릴러 드라마 장르만의  긴장감, 몰입감 굿   

10  ,  이래서 가정교육이 중요함..   

10  ,  상상했던 원작 느낌 그대로!!!   

10  ,  장동건 너무 연기 잘함~ 그동안 얼굴에 가린 연기력이 제대로 나오는 영화   

8  ,  사람이 그냥 악해지는게 아님~ 다 이유가 있는법   

10  ,  장동건.류승룡 연기 짱이고 내용도 괜찮고요   

10  ,  연기력 대박이네요 역시 배우들 클래스  

### 이제 크롤링한 내용을 텍스트파일로 내보내보자!

In [13]:
url1 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='
movies = [85578, 85579, 99354, 123519, 125494, 134963, 136898, 137008, 137326, 144379, 151728, 153620, 154449, 154598, 158626, 158885, 161242, 165026, 168298, 172454]
### 7년의 밤, 신과함께, 지슬, 아가씨, 퍼시빅림, 라라랜드, 레디플레이어원, 리얼, 블랙팬서, 러빙빈센트, 
### 코코, 치인트, 리틀포레스트, 그레이50, 셰잎옾워터, 콜미바이유어네임, 범죄도시, 사라진밤, 지금만나러갑니다, 곤지암
url2 = '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='


for i in movies:
    f = open('Movie_'+str(i)+'.txt', 'w')   ### 영화 한개 마다 txt 파일로 저장
    num = i
    response = req.urlopen('https://movie.naver.com/movie/bi/mi/point.nhn?code='+str(num))
    soup = BeautifulSoup(response, 'html.parser')
    title = soup.select_one('title').string
    f.write('### '+title+' ###'+'\n\n')   ### 영화 제목 출력
    
    response = req.urlopen(url1+str(num)+url2+'1')   ### 댓글창 개수를 뽑아내보자
    soup = BeautifulSoup(response, 'html.parser')    ### 소스 파싱
    score_total = soup.find('div', class_ = 'score_total')    ### 평점 개수 나타난 곳 추출
    limit = score_total.find_all('em')[1]   ### 평점 개수 추출
    page_limit = int(limit.string.replace(',', ''))/10   ### 숫자형식으로 바꿔서 10으로 나눠줌
    page_limit = math.ceil(page_limit)   ### 소숫점 올림

    for j in range(1, page_limit):
        page = j
        response = req.urlopen(url1+str(num)+url2+str(page))
        soup = BeautifulSoup(response, 'html.parser')
        score_result = soup.find('div', class_ = 'score_result')
        lis = score_result.find_all('li')   # 리뷰가 담겨있는 각 마디를 잘라 벡터화
        
        for li in lis:
            score = li.select_one('div.star_score > em')
            reple = li.select_one('div.score_reple > p')
            if reple.find('span')!=None:
                reple.find('span').decompose()
            f.write(score.get_text()+' , '+reple.get_text()+'\n')   ### 평점, 리뷰 출력
    f.close   ### 텍스트 입력 종료

### 대략 2시간동안 22만개 출력  (1분에 2000개 꼴..)